In [1]:
# Carregando as libs necessarias
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import glob, os
# Importando as funcoes do arquivo functions.py
import functions 

In [2]:
# Define os locais de origem e destino dos arquivos
arq_ori=['./DataEngTest-main/raw_data/ads.zip','./DataEngTest-main/raw_data/buildings.zip']
arq_dest='./DataEngTest-main/raw_data/'
pos =0
loc_arquivos=[]

# Executa rotina para extrair os arquivos zipados 
for file in arq_ori:
    # chama funcao para extração dos arquivos
    path_extraido = functions.extrair_arquivo(file,arq_dest)
    pos+=1
    loc_arquivos.append(path_extraido)
    print(path_extraido)


./DataEngTest-main/raw_data/ads/
./DataEngTest-main/raw_data/buildings/


### Concat Json Files

In [3]:
# seleciona local dos arquivos json
loc_json= loc_arquivos[0]

path_json = os.path.join(loc_json,'*.json')
# faz a leitura dos diretorios
files= glob.glob(path_json)

In [4]:
# Rotina para fazer a leitura dos diretorios dos arquivos json e em seguida 
# grava todos os arquivos no dataframe df_ads

# define lista
ads=[]

for f in files:
    df_json =pd.read_json(f,lines=True, dtype={'street_number':str})
   # adiciona DataFrame de cada arquivo a lista "ads"
    ads.append(df_json)    
# Cria DataFrame
df_ads= pd.concat(ads, ignore_index=True)

In [7]:
# local do arquivo csv
loc_csv= loc_arquivos[1]
# leitura arquivo buildings
df_buildings = pd.read_csv(loc_csv+'buildings.csv',
                           dtype={'street_number':'str'},na_values="None",sep=',')

In [8]:
df_ads.head(2)

,bathrooms_min,bedrooms_min,built_area_min,city_name,idno,lat,lon,neighborhood,parking_space_min,property_type,sale_price,state,street,street_number
0,2.0,2.0,NaN,São Paulo,AP6909-INC,NaN,NaN,Vila Mariana,1.0,APARTMENT,1105100.0,SP,Rua Doutor Diogo de Faria,421
1,3.0,3.0,NaN,São Paulo,AP6166-INC,NaN,NaN,Brooklin,2.0,APARTMENT,NaN,SP,Rua California,1127


In [9]:
df_buildings.head(2)

,id,address,address_number,neighborhood,city,state,cep,latitude,longitude
0,2261901,Alameda Tabajara,534,Caixa D´Água,Vinhedo,São Paulo,1.32826e+07,-23.038020,-46.916604
1,390564,Avenida Afrânio de Melo Franco,20,Leblon,Rio de Janeiro,Rio de Janeiro,2.24301e+07,-22.985698,-43.217262


## Criação das tabelas no Banco

In [10]:
# Executa funcao para criação das tabelas
functions.create_tables()

#### Verificando qtd de registros por estado em ambos dataframes

In [11]:
# Qtd de registros por estado e cidade em "ADs"
df_ads.groupby(['state','city_name']).count()[['idno']]

,,idno
state,city_name,
SP,São Paulo,10000


In [12]:
# Qtd de registros por estado em "Buildings"
df_buildings.groupby(['state']).count()[['id']]

,id
state,
AC,8
AL,865
AM,205
AP,2
Acre,2
Alagoas,2169
Amapá,130
Amazonas,1782
BA,3192


In [13]:
# Vamos padronizar os registros "state" para sigla dos estados 
# Como exemplo, vamos atualizar somente a sigla "SP"
df_buildings.loc[(df_buildings['state']=='São Paulo','state')]='SP'

In [14]:
# Verificando qtd de registros depois da atualização
df_buildings.loc[(df_buildings['state']=='São Paulo') | (df_buildings['state']=='SP' )]\
.groupby('state').count()[['id']]

,id
state,
SP,253550


#### Verificação nomenclatura da cidade de "São Paulo"

In [15]:
df_buildings.query('city.str.contains("Paulo")',engine='python').groupby('city').count()[['id']]

,id
city,
Engenheiro Paulo de Frontin,1
Monsenhor Paulo,1
Paulo Afonso,12
Paulo Lopes,12
Paulo de Faria,2
Sao Paulo,16279
São Paulo,92085
São Paulo,4


In [16]:
# Retirando os espaços adicionais
df_buildings['city']= df_buildings['city'].str.rstrip()
df_buildings['city']= df_buildings['city'].str.lstrip()

In [17]:
# atualliza o nome da cidade para "São Paulo"
df_buildings.loc[(df_buildings['city']=='Sao Paulo','city')]='São Paulo'

In [18]:
# chama a função para receber os parametros de conexao
params = functions.config()
# define os parametros do banco
string_conn = 'postgresql://{user}:{passwd}@{host}:{port}/{db}'.format(
    user=params['user'], passwd=params['password'], host=params['host']
    , port=5432, db=params['database'])

alchemyEngine = create_engine(string_conn, pool_size = 50)

# Realiza a conexao
dbConn = alchemyEngine.connect();


In [19]:
# Carga das tabelas no BD
# com o comando "to_sql"
# tabela "ads" -> recebe dados de "dfs_ads" 
df_ads.to_sql('ads',con=dbConn,if_exists='append',index_label='id')

# tabela "BUILDINGS" -> recebe dados de df_buildings
df_buildings.to_sql('buildings',con=dbConn,if_exists='append',index=False)

#### Queries
##### No codigo abaixo há 3 queries:
##### A primeira query atualiza "id_building" com o "id" da tabela "buildings" utilizando os dados de logradouro, numero, bairro, cidade e estado.
##### Na segunda query, a atualizacao é feita por logradouro, numero, bairro, cidade e estado.
##### A terceira query, é feito o join de 1xN (1 registro de ad para "n" registros de buildings). O resultado é inserido na tabela "ad_buildings"

In [20]:
# faz a conexao 
conn = psycopg2.connect(**params)
cur = conn.cursor()
# Queries
# 1ª query -> atualiza o id_building na tabela ADs
# 2ª query -> atualiza o od_building restante
# 3ª query -> insere os registros com mais de um registro buildings encontrado na tabela ads_building
queries = ('''
update ADS set id_building = b.id 
from "buildings" b
where ads.street = b.address  
	and ads.street_number = b.address_number
    and ads.city_name = b.city
    and ads.state = b.state
    and ads.neighborhood =b.neighborhood 
''',
'''
-- atualizando utilizandos as inf de endereço sem "neighborhood"
-- para esses casos, como podemos encontrar mais de um registro de building para cada ad
-- vamos selecionar somente os registros com 1 building para 1 ad  

with cte as (select ads.id,count(b.id)as qtd
from ads , "buildings" b
where ads.street = b.address  
	and ads.street_number = b.address_number
    and ads.city_name = b.city
    and ads.state = b.state
    and ads.id_building is null -- Exclui os registros já encontrados
group by ads.id
having(count(b.id))= 1 --somente 1 registro por ad
order by ads.id)

-- Update
update ads set id_building= b.id
from buildings b
where ads.street = b.address  
	and ads.street_number = b.address_number
    and ads.city_name = b.city
    and ads.state = b.state
-- selecionando os casos retornados pelo CTE
and ads.id in (select id from cte)

''',
'''
-- Insere os registros com mais de 1 predio para um unico registro de ad na tabela ads_buildings
INSERT INTO ad_buildings (id_ad,id_build)
select a.id as id_ad, b.id as id_build
from ADS  a
inner join buildings b on a.street = b.address  
        and a.street_number = b.address_number
        and a.city_name = b.city
        and a.state = b.state                      
where id_building is null
'''        )

for query in queries:
    
    # Executa a query
    cur.execute(query)

# fecha conexao
cur.close()
# commit the changes
conn.commit()
if conn is not None:
    conn.close()


## Queries

#### A solução nao foi capaz de encontrar um registro de predio para cada registro de "ad"
##### O dataset de ad contem 10000 registros. Foram encontrados 8025 registros com um registro associado no dataset "buildings" pois possuem o mesmo endereço, numero cidade, estado e em alguns casos tambem o bairro. Esses registros estoa identificados com a informação de "id_building" na tabela "ads"
##### Há 539 registros de "ads", que foram associados a mais de um registro na tabela "buildings". Esses registros estão na tabela ads_building.
##### No dataset "ads", há 1436 registros que nao foram localizados no dataset "buidlings".

#### Abaixo, alguns pontos visando melhorar a associação dos registros de ads e buildings:

##### Atualizar as informaçoes referentes a lat e lon para os 2 datasets, com uma api de geolocalizacao, utilizando os dados de endereços
##### Padronizar as variaveis como "address", "neighborhood", "city" e "cep".
##### Atualizar os registros de cep para corrigir as informaçoes como bairro por exemplo, para evitar registros de predios com mesmo endereço e bairros diferentes. 
##### atualizar as informaçoes de "address","address_number" atraves das variaveis de "lat" e "lon"
##### algumas ferramentas:
##### "unidecode" -> para retirada de acentuação
##### "googlemaps" -> para busca de informações em relação ao endereço e geolocalização

Importante destacar que para o dataset de ads a informaçao do bairro muitas vezes nao é a mesma que está cadastrada nos correios. Acredito que isso aconteça devido aos anunciantes informarem o nome da região mais conhecida.
